In [1]:
# ============================================================================
# LEAK DETECTION HEURISTIC PIPELINE
# ============================================================================
# Detects and handles answer leaks in code solutions.
# - Only processes records with tier in {"core", "extend", "extended"}
# - Skips records already marked as "drop"
# - Uses heuristic detection + LLM confirmation + repair pipeline
# - Outputs same number of records, modified in-place
# ============================================================================

import json
import sys
import re
import io
import traceback
from pathlib import Path
from tqdm.notebook import tqdm
from datetime import datetime
from typing import Dict, Any, Optional, List, Tuple, Set
from collections import Counter
import time
from contextlib import redirect_stdout, redirect_stderr
import multiprocessing

import anyio
import httpx
from llm_pool import LLMPool
from runtime_config import RuntimeConfig

In [2]:
# ============================================================================
# CONFIGURATION PARAMETERS
# ============================================================================

# --- LLM Configuration ---
API_KEY = "sk-local"
MODEL_NAME = "gpt-oss"
NGINX_BALANCER_URL = "http://127.0.0.1:8080/v1"

TEMPERATURE = 0.2
TOP_P = 0.95
MAX_TOKENS = 1024 * 4  # Need more tokens for code repair
REASONING_EFFORT = "low"

# --- Input/Output Configuration ---
INPUT_JSONL_PATH = Path("/home/larcanio/AIMO3_v2/data/datasets/Dataset_Full/dataset_raw.jsonl")
OUTPUT_JSONL_PATH = INPUT_JSONL_PATH.parent / "dataset_leak_cleaned.jsonl"

# --- Processing Configuration ---
MAX_PROBLEMS_TO_PROCESS = None  # Set to None to process all
START_FROM_INDEX = 0

# --- Tier Filter Configuration ---
PROCESSABLE_TIERS = {"core", "extend", "extended"}
DROP_TIERS = {"drop", "dropped"}

# --- Hot-reloadable Runtime Configuration ---
CONFIG_FILE = "config_leak_detection.json"

cfg = RuntimeConfig(CONFIG_FILE, defaults={
    "LLM_REQUEST_RETRY_COUNT": 3,
    "LLM_REQUEST_TIMEOUT_SECONDS": 300,
    "MAX_CONCURRENT_REQUESTS": 25,
})

# --- Code Execution Sandbox Settings ---
EXECUTION_TIMEOUT_SECONDS = 30

print("Configuration loaded:")
print(f"  Model: {MODEL_NAME}")
print(f"  Config file: {CONFIG_FILE}")
print(cfg)
print(f"  Input file: {INPUT_JSONL_PATH}")
print(f"  Output file: {OUTPUT_JSONL_PATH}")
print(f"  Max problems: {MAX_PROBLEMS_TO_PROCESS or 'All'}")
print(f"  Processable tiers: {PROCESSABLE_TIERS}")

Configuration loaded:
  Model: gpt-oss
  Config file: config_leak_detection.json
RuntimeConfig(LLM_REQUEST_RETRY_COUNT=3, LLM_REQUEST_TIMEOUT_SECONDS=300, MAX_CONCURRENT_REQUESTS=25)
  Input file: /home/larcanio/AIMO3_v2/data/datasets/Dataset_Full/dataset_raw.jsonl
  Output file: /home/larcanio/AIMO3_v2/data/datasets/Dataset_Full/dataset_leak_cleaned.jsonl
  Max problems: All
  Processable tiers: {'extended', 'core', 'extend'}


In [3]:
# ============================================================================
# PROMPT TEMPLATES
# ============================================================================

# --- PASS 1: Leak Confirmation Classifier ---
LEAK_CLASSIFIER_SYSTEM_PROMPT = """# ROLE
You are a code auditor checking for answer leaks in math competition solutions.

An "answer leak" is when the expected answer appears directly in the code in a way
that reveals it was not properly derived from the problem.

# CONTEXT
You will receive:
- problem_text: the math problem statement
- expected_answer: the correct integer answer
- code: the solution code being audited
- heuristic_bucket: what pattern triggered this review
- answer_in_problem: whether the expected answer appears in the problem text

# OUTPUT
Return ONLY valid JSON with this exact structure:

{
  "label": "<one of: no_leak, leak_print_or_comment, leak_assert_or_hardcode, leak_answer_seeded, uncertain>",
  "confidence": 0.0-1.0,
  "reasons": ["<short reasons>"],
  "evidence": [{"type": "code|comment", "snippet": "..."}]
}

# LABEL DEFINITIONS

## no_leak
The answer token appears but is NOT a leak:
- The answer is given in the problem as input data
- The answer appears as an incidental intermediate value during correct computation
- The code properly computes and derives the answer from problem givens

## leak_print_or_comment
The code directly outputs or mentions the expected answer:
- print(ANSWER) or return ANSWER as a literal constant
- Comments state "answer is ANSWER" or "result = ANSWER"
- The expected answer is documented in comments as the final result

## leak_assert_or_hardcode
The expected answer is hardcoded or asserted:
- assert computed == ANSWER (checking against known answer)
- ans = ANSWER without derivation
- return ANSWER without computation
- The answer is assigned directly rather than computed

## leak_answer_seeded
The expected answer is used as an input seed in computation:
- x = ANSWER * k (using answer as factor)
- for i in range(ANSWER) (using answer as bound)
- The computation relies on knowing the answer beforehand

## uncertain
Ambiguous cases where you cannot determine if it's a leak.

# RULES

1. Structural asserts are NOT leaks:
   - assert n > 0 (positivity check)
   - assert gcd(a, b) == 1 (coprimality)
   - assert result % 2 == 0 (parity check)
   These don't force the final value unless they literally assert == ANSWER.

2. If answer_in_problem is True, the answer appearing in code may be legitimate
   (e.g., it's input data). Check if it's used as input vs used as the final result.

3. Look at the FINAL value that is printed/returned. If the computation derives
   this value step by step from the problem, it's not a leak.

4. Be conservative: if clearly a leak, label it. If unclear, use uncertain.

# HARD CONSTRAINTS
- JSON only, no comments or explanations outside JSON
- No additional keys beyond those specified
"""

LEAK_CLASSIFIER_USER_PROMPT = """Problem:
<<<
{PROBLEM_TEXT}
>>>

Expected Answer (integer): {EXPECTED_ANSWER}

Code:
<<<
{CODE}
>>>

Heuristic bucket: {HEURISTIC_BUCKET}
Answer appears in problem text: {ANSWER_IN_PROBLEM}

Analyze this code and return ONLY valid JSON."""

# --- PASS 2: Repair Prompt ---
REPAIR_SYSTEM_PROMPT = """# ROLE
You are a code repair specialist. You must fix a solution that contains an answer leak.

# TASK
Rewrite the code to compute the answer from the problem's given information,
WITHOUT using the expected answer value anywhere in the code.

# OUTPUT
Output PYTHON CODE ONLY:
- No markdown code fences
- No explanations or comments mentioning the answer
- Must be a complete, runnable solution
- Must print the final answer (integer)
- Must NOT contain the literal expected answer anywhere
- Must NOT include asserts against the expected answer
- Must compute from problem givens
- Keep deterministic; no external I/O
- Prefer integers; avoid floating point when possible

If repair is impossible (problem doesn't have enough information to derive answer),
output EXACTLY: REPAIR_NOT_POSSIBLE

# HARD CONSTRAINTS
- Output ONLY Python code or REPAIR_NOT_POSSIBLE
- No markdown, no explanations
- The forbidden answer value must NOT appear anywhere in output
"""

REPAIR_USER_PROMPT = """Problem:
<<<
{PROBLEM_TEXT}
>>>

Forbidden answer value (DO NOT USE THIS IN YOUR CODE): {EXPECTED_ANSWER}

Original leaked code (for reference of what NOT to do):
<<<
{CODE}
>>>

Write a repaired solution that computes the answer from the problem without using the forbidden value.
Output ONLY Python code (no markdown) or REPAIR_NOT_POSSIBLE."""

print("Prompts loaded")

Prompts loaded


In [4]:
# ============================================================================
# HELPER FUNCTIONS
# ============================================================================

def parse_json_from_response(text: str) -> Dict[str, Any]:
    """Extract and parse JSON from LLM response."""
    if not text or text.strip() == "":
        raise ValueError("Empty response from LLM")
    
    text = text.strip()
    
    # Remove markdown code blocks if present
    if "```" in text:
        code_block_pattern = r"```(?:json)?\s*\n?(.*?)```"
        matches = re.findall(code_block_pattern, text, re.DOTALL)
        if matches:
            text = matches[0].strip()
    
    # Find JSON object
    start_idx = text.find("{")
    if start_idx == -1:
        raise ValueError("No JSON object found in response")
    
    # Find matching closing brace
    brace_count = 0
    end_idx = start_idx
    for i in range(start_idx, len(text)):
        if text[i] == "{":
            brace_count += 1
        elif text[i] == "}":
            brace_count -= 1
            if brace_count == 0:
                end_idx = i + 1
                break
    
    json_str = text[start_idx:end_idx]
    
    try:
        return json.loads(json_str)
    except json.JSONDecodeError as e:
        # Fix trailing commas
        json_str_fixed = re.sub(r",\s*}", "}", json_str)
        json_str_fixed = re.sub(r",\s*]", "]", json_str_fixed)
        try:
            return json.loads(json_str_fixed)
        except:
            raise ValueError(f"Failed to parse JSON: {e}. Text: {json_str[:500]}")


def extract_code_from_response(text: str) -> str:
    """Extract Python code from LLM response, handling markdown fences."""
    if not text:
        return ""
    
    text = text.strip()
    
    # Check for REPAIR_NOT_POSSIBLE
    if "REPAIR_NOT_POSSIBLE" in text:
        return "REPAIR_NOT_POSSIBLE"
    
    # Remove markdown code fences
    if "```" in text:
        code_block_pattern = r"```(?:python)?\s*\n?(.*?)```"
        matches = re.findall(code_block_pattern, text, re.DOTALL)
        if matches:
            return matches[0].strip()
    
    return text.strip()


def build_answer_pattern(answer: int) -> re.Pattern:
    """Build regex pattern for exact integer token match."""
    # Handle negative numbers
    answer_str = str(answer)
    if answer < 0:
        # Match -123 but not --123 or 1-123
        return re.compile(r'(?<![\d-])' + re.escape(answer_str) + r'(?!\d)')
    else:
        # Match 123 but not 1234 or 0123
        return re.compile(r'(?<!\d)' + re.escape(answer_str) + r'(?!\d)')


def extract_comments(code: str) -> str:
    """Extract all comments from Python code."""
    comments = []
    
    # Line comments: # ...
    line_comments = re.findall(r'#.*', code)
    comments.extend(line_comments)
    
    # Triple-quoted strings (''' or \"\"\")
    triple_single = re.findall(r"'''[\s\S]*?'''", code)
    triple_double = re.findall(r'"""[\s\S]*?"""', code)
    comments.extend(triple_single)
    comments.extend(triple_double)
    
    return "\n".join(comments)


def check_direct_print_or_return(code: str, answer: int) -> bool:
    """Check if code directly prints or returns the literal expected answer."""
    answer_str = str(answer)
    
    # Pattern for print(ANSWER) or print(ANSWER, ...)
    print_patterns = [
        rf'print\s*\(\s*{re.escape(answer_str)}\s*\)',
        rf'print\s*\(\s*{re.escape(answer_str)}\s*,',
        rf'print\s*\(\s*{re.escape(answer_str)}\s*\)',
    ]
    
    # Pattern for return ANSWER
    return_patterns = [
        rf'return\s+{re.escape(answer_str)}(?!\d)',
        rf'return\s*\(\s*{re.escape(answer_str)}\s*\)',
    ]
    
    for pattern in print_patterns + return_patterns:
        if re.search(pattern, code):
            return True
    
    return False


def check_assert_equals(code: str, answer: int) -> bool:
    """Check if code contains assert ... == ANSWER."""
    answer_str = str(answer)
    patterns = [
        rf'assert\s+.*==\s*{re.escape(answer_str)}(?!\d)',
        rf'assert\s+{re.escape(answer_str)}\s*==',
    ]
    for pattern in patterns:
        if re.search(pattern, code):
            return True
    return False


def _code_execution_worker(code: str, result_queue: multiprocessing.Queue) -> None:
    """
    Worker function that runs in a separate process.
    Executes code and puts result into the queue.
    """
    stdout_capture = io.StringIO()
    stderr_capture = io.StringIO()
    
    # Create isolated namespace
    namespace = {
        '__builtins__': __builtins__,
        '__name__': '__main__',
        '__doc__': None,
    }
    
    try:
        with redirect_stdout(stdout_capture), redirect_stderr(stderr_capture):
            exec(code, namespace)
        result_queue.put((True, stdout_capture.getvalue(), stderr_capture.getvalue()))
    except Exception as e:
        tb = traceback.format_exc()
        result_queue.put((False, stdout_capture.getvalue(), tb))


def execute_code_safely(code: str, timeout: int = None) -> Tuple[bool, str, str]:
    """
    Execute Python code safely with timeout using multiprocessing.Process.
    
    This implementation properly terminates runaway processes by using
    Process.terminate() and Process.kill() instead of relying on
    ProcessPoolExecutor's cooperative cancellation.
    
    Returns: (success, stdout, stderr)
    """
    if timeout is None:
        timeout = EXECUTION_TIMEOUT_SECONDS
    
    # Use 'spawn' context for cleaner subprocess isolation
    # This avoids issues with fork() in threaded environments like Jupyter
    ctx = multiprocessing.get_context('spawn')
    result_queue = ctx.Queue()
    
    proc = ctx.Process(
        target=_code_execution_worker,
        args=(code, result_queue),
        daemon=True  # Ensure process is killed if parent dies
    )
    
    try:
        proc.start()
        proc.join(timeout=timeout)
        
        if proc.is_alive():
            # Process is still running - force terminate it
            proc.terminate()
            proc.join(timeout=2)  # Give it 2 seconds to terminate gracefully
            
            if proc.is_alive():
                # Still alive after SIGTERM - use SIGKILL
                proc.kill()
                proc.join(timeout=1)
            
            # Drain the queue to prevent resource leaks
            try:
                while not result_queue.empty():
                    result_queue.get_nowait()
            except:
                pass
            
            return False, "", f"Execution timed out after {timeout} seconds"
        
        # Process completed - get result from queue
        if not result_queue.empty():
            try:
                result = result_queue.get(timeout=1)
                return result
            except:
                return False, "", "Failed to retrieve result from worker process"
        else:
            # Process exited but no result in queue - likely crashed
            exit_code = proc.exitcode
            if exit_code != 0:
                return False, "", f"Worker process exited with code {exit_code}"
            return False, "", "Worker process completed but produced no output"
    
    except Exception as e:
        # Ensure process is cleaned up on any error
        if proc.is_alive():
            proc.terminate()
            proc.join(timeout=1)
            if proc.is_alive():
                proc.kill()
                proc.join(timeout=1)
        return False, "", f"Execution error: {type(e).__name__}: {str(e)}"
    
    finally:
        # Final cleanup - ensure process is dead and queue is closed
        if proc.is_alive():
            proc.kill()
            proc.join(timeout=1)
        
        try:
            result_queue.close()
            result_queue.join_thread()
        except:
            pass


def add_risk_flag(record: Dict[str, Any], flag: str) -> None:
    """Add a risk flag to a record, avoiding duplicates."""
    if "risk_flags" not in record:
        record["risk_flags"] = []
    if flag not in record["risk_flags"]:
        record["risk_flags"].append(flag)

print("Helper functions loaded")

Helper functions loaded


In [5]:
# ============================================================================
# LOAD DATA FROM JSONL
# ============================================================================

def load_records_from_jsonl(jsonl_path: Path) -> List[Dict[str, Any]]:
    """Load all records from JSONL file."""
    records = []
    with open(jsonl_path, 'r', encoding='utf-8') as f:
        for line_num, line in enumerate(f, 1):
            if line.strip():
                try:
                    data = json.loads(line)
                    records.append(data)
                except json.JSONDecodeError as e:
                    print(f"Error parsing line {line_num}: {e}")
                    continue
    return records


def validate_record_schema(record: Dict[str, Any]) -> bool:
    """Validate that record has required fields."""
    try:
        _ = record["problem"]["text"]
        _ = record["problem"]["expected_answer"]
        _ = record["attempts"]
        _ = record["outcome"]["pass_at_k"]
        return True
    except (KeyError, TypeError):
        return False


# Load records
all_records = load_records_from_jsonl(INPUT_JSONL_PATH)
print(f"Loaded {len(all_records)} records from {INPUT_JSONL_PATH}")

# Validate schema
valid_count = sum(1 for r in all_records if validate_record_schema(r))
print(f"Schema valid: {valid_count}/{len(all_records)}")

# Apply limits if configured
if START_FROM_INDEX > 0:
    print(f"Starting from index {START_FROM_INDEX}")
if MAX_PROBLEMS_TO_PROCESS:
    print(f"Limiting to {MAX_PROBLEMS_TO_PROCESS} problems")

Loaded 68191 records from /home/larcanio/AIMO3_v2/data/datasets/Dataset_Full/dataset_raw.jsonl
Schema valid: 68191/68191


In [6]:
# ============================================================================
# STAGE 1 - HEURISTIC LEAK CANDIDATE DETECTION
# ============================================================================

def classify_heuristic_bucket(code: str, comment_text: str, answer: int, answer_in_problem: bool) -> Optional[str]:
    """
    Determine the heuristic bucket for a candidate.
    Returns None if not a candidate, otherwise returns the bucket name.
    Priority order: (a) direct_print_or_return, (b) assert_equals, (c) comment_mentions, (d) other
    """
    pattern = build_answer_pattern(answer)
    
    # Check if answer token appears in code at all
    if not pattern.search(code):
        return None
    
    # (a) direct_print_or_return - AUTO-DROP
    if check_direct_print_or_return(code, answer):
        return "direct_print_or_return"
    
    # (b) assert_equals
    if check_assert_equals(code, answer):
        return "assert_equals"
    
    # (c) comment_mentions (only if answer NOT in problem text)
    if not answer_in_problem and pattern.search(comment_text):
        return "comment_mentions"
    
    # (d) other - answer appears elsewhere in code
    return "other"


def run_heuristic_detection(records: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Run heuristic detection on all records.
    Returns statistics and marks candidates in-place.
    """
    stats = {
        "total_records": len(records),
        "skipped_missing_tier": 0,
        "skipped_drop_tier": 0,
        "skipped_non_processable_tier": 0,
        "skipped_invalid_schema": 0,
        "skipped_no_passing_attempt": 0,
        "processed": 0,
        "candidates": 0,
        "auto_drop_direct_print": 0,
        "buckets": Counter(),
        "datasets": Counter(),
    }
    
    candidates = []  # (record_idx, record, bucket, code, answer, problem_text, answer_in_problem)
    
    for idx, record in enumerate(tqdm(records, desc="Heuristic detection")):
        # Get tier
        tier = record.get("tier") or record.get("audit", {}).get("tier")
        
        # Tier filter
        if tier is None:
            stats["skipped_missing_tier"] += 1
            continue
        
        tier_lower = tier.lower() if isinstance(tier, str) else ""
        
        if tier_lower in DROP_TIERS:
            stats["skipped_drop_tier"] += 1
            continue
        
        if tier_lower not in {t.lower() for t in PROCESSABLE_TIERS}:
            stats["skipped_non_processable_tier"] += 1
            continue
        
        # Schema validation
        if not validate_record_schema(record):
            stats["skipped_invalid_schema"] += 1
            continue
        
        # Get passing attempt code
        pass_at_k = record["outcome"].get("pass_at_k") or 0
        attempts = record.get("attempts", [])
        
        if pass_at_k <= 0 or pass_at_k > len(attempts):
            stats["skipped_no_passing_attempt"] += 1
            continue
        
        code = attempts[pass_at_k - 1].get("code", "")
        if not code:
            stats["skipped_no_passing_attempt"] += 1
            continue
        
        stats["processed"] += 1
        
        # Get answer and problem text
        try:
            answer = int(record["problem"]["expected_answer"])
        except (ValueError, TypeError):
            # Non-integer answer - skip leak detection
            continue
        
        problem_text = record["problem"]["text"]
        dataset = record.get("dataset", "unknown")
        
        # Check if answer appears in problem
        answer_pattern = build_answer_pattern(answer)
        answer_in_problem = bool(answer_pattern.search(problem_text))
        
        # Extract comments
        comment_text = extract_comments(code)
        
        # Classify bucket
        bucket = classify_heuristic_bucket(code, comment_text, answer, answer_in_problem)
        
        if bucket is None:
            # Not a candidate - answer doesn't appear in code
            continue
        
        stats["candidates"] += 1
        stats["buckets"][bucket] += 1
        stats["datasets"][dataset] += 1
        
        # Handle auto-drop for direct_print_or_return
        if bucket == "direct_print_or_return":
            stats["auto_drop_direct_print"] += 1
            # Mark for auto-drop but don't process with LLM
            record["leak_audit"] = {
                "heuristic_bucket": bucket,
                "answer_in_problem": answer_in_problem,
                "auto_drop": True,
                "pass1": {
                    "label": "leak_print_or_comment",
                    "confidence": 1.0,
                    "reasons": ["Direct print/return of literal expected answer detected by heuristic"],
                    "evidence": [{"type": "code", "snippet": f"print({answer}) or return {answer}"}]
                }
            }
            # Set tier to drop
            record["tier"] = "drop"
            add_risk_flag(record, "answer_leak_direct_print")
        else:
            # Add to candidates for LLM processing
            candidates.append({
                "idx": idx,
                "record": record,
                "bucket": bucket,
                "code": code,
                "answer": answer,
                "problem_text": problem_text,
                "answer_in_problem": answer_in_problem,
                "pass_at_k": pass_at_k,
            })
    
    return {
        "stats": stats,
        "candidates": candidates,
    }


# Run heuristic detection
heuristic_result = run_heuristic_detection(all_records)
heuristic_stats = heuristic_result["stats"]
candidates = heuristic_result["candidates"]

print("\n" + "="*70)
print("HEURISTIC DETECTION RESULTS")
print("="*70)
print(f"Total records: {heuristic_stats['total_records']}")
print(f"Skipped (missing tier): {heuristic_stats['skipped_missing_tier']}")
print(f"Skipped (drop tier): {heuristic_stats['skipped_drop_tier']}")
print(f"Skipped (non-processable tier): {heuristic_stats['skipped_non_processable_tier']}")
print(f"Skipped (invalid schema): {heuristic_stats['skipped_invalid_schema']}")
print(f"Skipped (no passing attempt): {heuristic_stats['skipped_no_passing_attempt']}")
print(f"Processed: {heuristic_stats['processed']}")
print(f"\nCandidates found: {heuristic_stats['candidates']}")
print(f"Auto-dropped (direct print/return): {heuristic_stats['auto_drop_direct_print']}")
print(f"Candidates for LLM review: {len(candidates)}")
print(f"\nBucket breakdown:")
for bucket, count in heuristic_stats['buckets'].most_common():
    print(f"  {bucket}: {count}")
print(f"\nDataset breakdown (top 10):")
for dataset, count in heuristic_stats['datasets'].most_common(10):
    print(f"  {dataset}: {count}")

Heuristic detection:   0%|          | 0/68191 [00:00<?, ?it/s]


HEURISTIC DETECTION RESULTS
Total records: 68191
Skipped (missing tier): 0
Skipped (drop tier): 13802
Skipped (non-processable tier): 0
Skipped (invalid schema): 0
Skipped (no passing attempt): 0
Processed: 54389

Candidates found: 16753
Auto-dropped (direct print/return): 2522
Candidates for LLM review: 14231

Bucket breakdown:
  comment_mentions: 6903
  other: 5236
  direct_print_or_return: 2522
  assert_equals: 2092

Dataset breakdown (top 10):
  mvidia_reasoning_steps: 9359
  numina1.5: 6180
  numina1.5:cn_contest: 362
  gsm8k: 269
  numina1.5:metamath: 253
  numina1.5:aops: 221
  numina1.5:inequalities: 88
  numina1.5:number_theory: 21


In [ ]:
# ============================================================================
# STAGE 2 - LLM PASS 1: LEAK CONFIRMATION
# ============================================================================

SAVE_EVERY = 500


def _save_checkpoint(completed_n, total_n, save_num):
    """Write all_records to disk atomically (runs in thread)."""
    OUTPUT_JSONL_PATH.parent.mkdir(parents=True, exist_ok=True)
    tmp = OUTPUT_JSONL_PATH.with_suffix('.tmp')
    with open(tmp, 'w', encoding='utf-8') as f:
        for r in all_records:
            f.write(json.dumps(r, ensure_ascii=False) + '\n')
    tmp.rename(OUTPUT_JSONL_PATH)
    print(f"\n[checkpoint {save_num}] {completed_n}/{total_n} saved")


async def classify_leak_llm(pool: LLMPool, candidate: Dict[str, Any]) -> Optional[Dict[str, Any]]:
    """
    Use LLM to classify whether the candidate is a leak.
    Returns the classification result or None on failure.
    """
    try:
        user_prompt = LEAK_CLASSIFIER_USER_PROMPT.format(
            PROBLEM_TEXT=candidate["problem_text"],
            EXPECTED_ANSWER=candidate["answer"],
            CODE=candidate["code"],
            HEURISTIC_BUCKET=candidate["bucket"],
            ANSWER_IN_PROBLEM=str(candidate["answer_in_problem"]).lower(),
        )
        
        messages = [
            {"role": "system", "content": LEAK_CLASSIFIER_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ]
        
        resp = await pool.request(
            messages,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            top_p=TOP_P,
            reasoning_effort=REASONING_EFFORT,
        )
        
        if not resp.content:
            return None
        
        result = parse_json_from_response(resp.content)
        result["_tokens"] = {"prompt": resp.prompt_tokens, "completion": resp.completion_tokens}
        return result
        
    except Exception as e:
        print(f"Error in LLM classification: {type(e).__name__}: {str(e)[:100]}")
        return None


async def run_pass1_all(candidates: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Run Pass 1 (leak classification) on all candidates.
    Uses gated spawn + two-level TG + atomic checkpoints.
    """
    if not candidates:
        print("No candidates to process in Pass 1")
        return {"labels": Counter()}
    
    write_lock = anyio.Lock()
    save_lock = anyio.Lock()
    save_count = 0
    counter = {
        "done": 0,
        "labels": {},
        "tokens_in": 0,
        "tokens_out": 0,
    }
    
    pbar = tqdm(total=len(candidates), desc="Pass 1: Leak classification")
    spawn_limit = anyio.Semaphore(cfg.MAX_CONCURRENT_REQUESTS * 3)

    async def config_reloader(pool):
        """Re-read config every 60s and apply changes to the pool."""
        while True:
            await anyio.sleep(60)
            cfg.reload()
            pool._limiter.total_tokens = cfg.MAX_CONCURRENT_REQUESTS
            pool.max_retries = cfg.LLM_REQUEST_RETRY_COUNT
            pool._client.timeout = httpx.Timeout(cfg.LLM_REQUEST_TIMEOUT_SECONDS, connect=30)

    async def process_candidate(pool, candidate):
        nonlocal save_count

        result = await classify_leak_llm(pool, candidate)
        
        record = candidate["record"]
        
        if result is None:
            result = {
                "label": "uncertain",
                "confidence": 0.0,
                "reasons": ["LLM classification failed"],
                "evidence": []
            }
        
        # Store result in record (mutates all_records in-place via reference)
        record["leak_audit"] = {
            "heuristic_bucket": candidate["bucket"],
            "answer_in_problem": candidate["answer_in_problem"],
            "pass1": result,
        }
        
        should_save = False
        async with write_lock:
            counter["done"] += 1
            label = result.get("label", "unknown")
            counter["labels"][label] = counter["labels"].get(label, 0) + 1
            counter["tokens_in"] += result.get("_tokens", {}).get("prompt", 0)
            counter["tokens_out"] += result.get("_tokens", {}).get("completion", 0)
            pbar.update(1)
            should_save = counter["done"] % SAVE_EVERY == 0

        if should_save:
            async with save_lock:
                save_count += 1
                await anyio.to_thread.run_sync(
                    _save_checkpoint, counter["done"], len(candidates), save_count,
                )

    async with LLMPool(
        base_url=NGINX_BALANCER_URL,
        api_key=API_KEY,
        model=MODEL_NAME,
        max_inflight=cfg.MAX_CONCURRENT_REQUESTS,
        timeout=cfg.LLM_REQUEST_TIMEOUT_SECONDS,
        max_retries=cfg.LLM_REQUEST_RETRY_COUNT,
    ) as pool:
        async with anyio.create_task_group() as tg:
            tg.start_soon(config_reloader, pool)
            async with anyio.create_task_group() as work_tg:
                for candidate in candidates:
                    await spawn_limit.acquire()
                    async def _run(pool=pool, c=candidate):
                        try:
                            await process_candidate(pool, c)
                        finally:
                            spawn_limit.release()
                    work_tg.start_soon(_run)
            tg.cancel_scope.cancel()
    
    pbar.close()
    
    # Final checkpoint
    save_count += 1
    _save_checkpoint(counter["done"], len(candidates), save_count)
    
    print(f"\nPass 1 completed: {counter['done']} candidates processed")
    print(f"Tokens: in={counter['tokens_in']:,}, out={counter['tokens_out']:,}")
    print("\nLabel distribution:")
    for label, count in sorted(counter["labels"].items()):
        print(f"  {label}: {count}")
    
    return counter


# Run Pass 1
pass1_result = await run_pass1_all(candidates)

In [ ]:
# ============================================================================
# STAGE 3 - LLM PASS 2: REPAIR (ONLY FOR REPAIRABLE LEAKS)
# ============================================================================

REPAIRABLE_LABELS = {"leak_assert_or_hardcode", "leak_answer_seeded"}


def get_repairable_candidates(candidates: List[Dict[str, Any]]) -> List[Dict[str, Any]]:
    """
    Filter candidates that need repair based on Pass 1 labels.
    """
    repairable = []
    for candidate in candidates:
        record = candidate["record"]
        leak_audit = record.get("leak_audit", {})
        pass1 = leak_audit.get("pass1", {})
        label = pass1.get("label", "")
        
        if label in REPAIRABLE_LABELS:
            repairable.append(candidate)
    
    return repairable


async def repair_code_llm(pool: LLMPool, candidate: Dict[str, Any]) -> Tuple[str, str]:
    """
    Use LLM to repair leaked code.
    Returns: (status, code) where status is "repaired" or "not_possible"
    """
    try:
        user_prompt = REPAIR_USER_PROMPT.format(
            PROBLEM_TEXT=candidate["problem_text"],
            EXPECTED_ANSWER=candidate["answer"],
            CODE=candidate["code"],
        )
        
        messages = [
            {"role": "system", "content": REPAIR_SYSTEM_PROMPT},
            {"role": "user", "content": user_prompt},
        ]
        
        resp = await pool.request(
            messages,
            temperature=TEMPERATURE,
            max_tokens=MAX_TOKENS,
            top_p=TOP_P,
            reasoning_effort=REASONING_EFFORT,
        )
        
        if not resp.content:
            return "not_possible", "REPAIR_NOT_POSSIBLE"
        
        code = extract_code_from_response(resp.content)
        
        if code == "REPAIR_NOT_POSSIBLE" or not code:
            return "not_possible", "REPAIR_NOT_POSSIBLE"
        
        return "repaired", code
        
    except Exception as e:
        print(f"Error in LLM repair: {type(e).__name__}: {str(e)[:100]}")
        return "not_possible", "REPAIR_NOT_POSSIBLE"


async def run_pass2_all(candidates: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Run Pass 2 (repair) on all repairable candidates.
    Uses gated spawn + two-level TG + atomic checkpoints.
    """
    repairable = get_repairable_candidates(candidates)
    
    if not repairable:
        print("No repairable candidates found")
        return {"repaired": 0, "not_possible": 0}
    
    print(f"Found {len(repairable)} repairable candidates")
    
    write_lock = anyio.Lock()
    save_lock = anyio.Lock()
    save_count = 0
    counter = {
        "done": 0,
        "repaired": 0,
        "not_possible": 0,
    }
    
    pbar = tqdm(total=len(repairable), desc="Pass 2: Code repair")
    spawn_limit = anyio.Semaphore(cfg.MAX_CONCURRENT_REQUESTS * 3)

    async def config_reloader(pool):
        """Re-read config every 60s and apply changes to the pool."""
        while True:
            await anyio.sleep(60)
            cfg.reload()
            pool._limiter.total_tokens = cfg.MAX_CONCURRENT_REQUESTS
            pool.max_retries = cfg.LLM_REQUEST_RETRY_COUNT
            pool._client.timeout = httpx.Timeout(cfg.LLM_REQUEST_TIMEOUT_SECONDS, connect=30)

    async def process_candidate(pool, candidate):
        nonlocal save_count

        status, code = await repair_code_llm(pool, candidate)
        
        record = candidate["record"]
        record["leak_audit"]["pass2_repair"] = {
            "status": status,
            "code": code,
        }
        
        should_save = False
        async with write_lock:
            counter["done"] += 1
            if status == "repaired":
                counter["repaired"] += 1
            else:
                counter["not_possible"] += 1
            pbar.update(1)
            should_save = counter["done"] % SAVE_EVERY == 0

        if should_save:
            async with save_lock:
                save_count += 1
                await anyio.to_thread.run_sync(
                    _save_checkpoint, counter["done"], len(repairable), save_count,
                )

    async with LLMPool(
        base_url=NGINX_BALANCER_URL,
        api_key=API_KEY,
        model=MODEL_NAME,
        max_inflight=cfg.MAX_CONCURRENT_REQUESTS,
        timeout=cfg.LLM_REQUEST_TIMEOUT_SECONDS,
        max_retries=cfg.LLM_REQUEST_RETRY_COUNT,
    ) as pool:
        async with anyio.create_task_group() as tg:
            tg.start_soon(config_reloader, pool)
            async with anyio.create_task_group() as work_tg:
                for candidate in repairable:
                    await spawn_limit.acquire()
                    async def _run(pool=pool, c=candidate):
                        try:
                            await process_candidate(pool, c)
                        finally:
                            spawn_limit.release()
                    work_tg.start_soon(_run)
            tg.cancel_scope.cancel()
    
    pbar.close()
    
    # Final checkpoint
    save_count += 1
    _save_checkpoint(counter["done"], len(repairable), save_count)
    
    print(f"\nPass 2 completed:")
    print(f"  Repaired: {counter['repaired']}")
    print(f"  Not possible: {counter['not_possible']}")
    
    return counter


# Run Pass 2
pass2_result = await run_pass2_all(candidates)

In [9]:
# ============================================================================
# STAGE 4 - RE-VALIDATION OF REPAIRED CODE
# ============================================================================

def validate_repaired_code(record: Dict[str, Any]) -> Tuple[bool, str]:
    """
    Validate that repaired code:
    1. Executes successfully (with timeout)
    2. Prints the expected answer
    3. Does not contain the answer token
    4. Does not directly print/return the literal answer
    
    Returns: (valid, reason)
    """
    leak_audit = record.get("leak_audit", {})
    pass2 = leak_audit.get("pass2_repair", {})
    
    if pass2.get("status") != "repaired":
        return False, "No repaired code available"
    
    repaired_code = pass2.get("code", "")
    if not repaired_code or repaired_code == "REPAIR_NOT_POSSIBLE":
        return False, "Repaired code is empty or not possible"
    
    # Get expected answer
    try:
        expected_answer = int(record["problem"]["expected_answer"])
    except (ValueError, TypeError, KeyError):
        return False, "Cannot parse expected answer"
    
    answer_pattern = build_answer_pattern(expected_answer)
    
    # Check 1: Answer token should not appear in repaired code
    if answer_pattern.search(repaired_code):
        return False, f"Repaired code still contains answer token {expected_answer}"
    
    # Check 2: No direct print/return of literal answer
    if check_direct_print_or_return(repaired_code, expected_answer):
        return False, "Repaired code directly prints/returns the answer"
    
    # Check 3: Execute and verify output (WITH TIMEOUT)
    success, stdout, stderr = execute_code_safely(repaired_code, timeout=EXECUTION_TIMEOUT_SECONDS)
    
    if not success:
        # Check if it was a timeout
        if "timed out" in stderr.lower():
            return False, f"Execution timed out after {EXECUTION_TIMEOUT_SECONDS}s"
        return False, f"Execution failed: {stderr[:200]}"
    
    # Check 4: Output matches expected answer
    stdout_clean = stdout.strip()
    if not stdout_clean:
        return False, "No output produced"
    
    try:
        # Try to extract last number from output
        output_lines = stdout_clean.split("\n")
        last_line = output_lines[-1].strip() if output_lines else ""
        
        # Try direct integer parse
        if last_line.lstrip("-").isdigit():
            output_answer = int(last_line)
        else:
            # Try to find integer in last line
            numbers = re.findall(r'-?\d+', last_line)
            if numbers:
                output_answer = int(numbers[-1])
            else:
                return False, f"Cannot parse output as integer: {last_line[:100]}"
        
        if output_answer != expected_answer:
            return False, f"Output {output_answer} does not match expected {expected_answer}"
        
    except ValueError:
        return False, f"Cannot parse output: {stdout_clean[:100]}"
    
    return True, "Validation passed"


def run_validation(candidates: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Validate all repaired candidates and update records accordingly.
    """
    stats = {
        "total_repaired": 0,
        "validation_passed": 0,
        "validation_failed": 0,
        "timeout_failures": 0,
        "failure_reasons": Counter(),
    }
    
    for candidate in tqdm(candidates, desc="Validating repairs"):
        record = candidate["record"]
        leak_audit = record.get("leak_audit", {})
        pass2 = leak_audit.get("pass2_repair", {})
        
        if pass2.get("status") != "repaired":
            continue
        
        stats["total_repaired"] += 1
        
        valid, reason = validate_repaired_code(record)
        leak_audit["validation"] = {
            "passed": valid,
            "reason": reason,
        }
        
        if valid:
            stats["validation_passed"] += 1
            
            # Replace code in-place
            pass_at_k = candidate["pass_at_k"]
            record["attempts"][pass_at_k - 1]["code"] = pass2["code"]
            add_risk_flag(record, "answer_leak_repaired")
            
        else:
            stats["validation_failed"] += 1
            
            # Track specific failure types
            if "timed out" in reason.lower():
                stats["timeout_failures"] += 1
            
            # Truncate reason for counter key
            reason_key = reason[:50] if len(reason) > 50 else reason
            stats["failure_reasons"][reason_key] += 1
            
            # Mark as drop
            record["tier"] = "drop"
            add_risk_flag(record, "answer_leak_repair_failed")
    
    print(f"\nValidation completed:")
    print(f"  Total repaired: {stats['total_repaired']}")
    print(f"  Passed: {stats['validation_passed']}")
    print(f"  Failed: {stats['validation_failed']}")
    print(f"  Timeout failures: {stats['timeout_failures']}")
    
    if stats["failure_reasons"]:
        print(f"\nFailure reasons (top 10):")
        for reason, count in stats["failure_reasons"].most_common(10):
            print(f"  {reason}: {count}")
    
    return stats


# Run validation
validation_result = run_validation(candidates)

Validating repairs:   0%|          | 0/14231 [00:00<?, ?it/s]

Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute '_code_execution_worker' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>
Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute '_code_execution_worker' on 


Validation completed:
  Total repaired: 3097
  Passed: 0
  Failed: 3097
  Timeout failures: 0

Failure reasons (top 10):
  Execution failed: Worker process exited with code : 2299
  Repaired code still contains answer token 1: 178
  Repaired code still contains answer token 2: 148
  Repaired code still contains answer token 0: 107
  Repaired code still contains answer token 3: 75
  Repaired code still contains answer token 4: 51
  Repaired code still contains answer token 5: 23
  Repaired code still contains answer token 6: 14
  Repaired code still contains answer token 10: 14
  Repaired code still contains answer token 8: 12


Traceback (most recent call last):
  File "<string>", line 1, in <module>
  File "/usr/lib/python3.12/multiprocessing/spawn.py", line 122, in spawn_main
    exitcode = _main(fd, parent_sentinel)
               ^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "/usr/lib/python3.12/multiprocessing/spawn.py", line 132, in _main
    self = reduction.pickle.load(from_parent)
           ^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^^
AttributeError: Can't get attribute '_code_execution_worker' on <module '__main__' (<class '_frozen_importlib.BuiltinImporter'>)>


In [10]:
# ============================================================================
# STAGE 5 - FINAL DECISIONS
# ============================================================================

def apply_final_decisions(candidates: List[Dict[str, Any]]) -> Dict[str, Any]:
    """
    Apply final tier decisions based on leak audit results.
    
    Decision logic:
    - direct_print_or_return => already tier="drop" (done in Stage 1)
    - pass1.label == leak_print_or_comment => tier="drop"
    - pass1.label == uncertain => keep, add "answer_leak_uncertain" flag
    - pass1.label == no_leak => keep unchanged
    - repaired and validated => keep, with "answer_leak_repaired"
    - unrepaired/not possible => tier="drop"
    """
    stats = {
        "no_leak_kept": 0,
        "uncertain_kept": 0,
        "leak_print_dropped": 0,
        "repaired_kept": 0,
        "unrepaired_dropped": 0,
        "repair_not_possible_dropped": 0,
    }
    
    for candidate in candidates:
        record = candidate["record"]
        leak_audit = record.get("leak_audit", {})
        pass1 = leak_audit.get("pass1", {})
        pass2 = leak_audit.get("pass2_repair", {})
        validation = leak_audit.get("validation", {})
        
        label = pass1.get("label", "")
        
        # Skip if already processed (auto-drop)
        if leak_audit.get("auto_drop"):
            continue
        
        # no_leak - keep unchanged
        if label == "no_leak":
            stats["no_leak_kept"] += 1
            continue
        
        # uncertain - keep but flag
        if label == "uncertain":
            stats["uncertain_kept"] += 1
            add_risk_flag(record, "answer_leak_uncertain")
            continue
        
        # leak_print_or_comment - drop
        if label == "leak_print_or_comment":
            stats["leak_print_dropped"] += 1
            record["tier"] = "drop"
            add_risk_flag(record, "answer_leak_print_or_comment")
            continue
        
        # Repairable labels - check repair status
        if label in REPAIRABLE_LABELS:
            repair_status = pass2.get("status", "")
            validation_passed = validation.get("passed", False)
            
            if repair_status == "repaired" and validation_passed:
                stats["repaired_kept"] += 1
                # Already handled in validation stage
                continue
            elif repair_status == "not_possible":
                stats["repair_not_possible_dropped"] += 1
                record["tier"] = "drop"
                add_risk_flag(record, "answer_leak_unrepaired")
            else:
                # Repair attempted but failed validation
                stats["unrepaired_dropped"] += 1
                # Already handled in validation stage
    
    print("\n" + "="*70)
    print("FINAL DECISION SUMMARY")
    print("="*70)
    print(f"Kept (no_leak): {stats['no_leak_kept']}")
    print(f"Kept (uncertain, flagged): {stats['uncertain_kept']}")
    print(f"Kept (repaired): {stats['repaired_kept']}")
    print(f"Dropped (leak_print_or_comment): {stats['leak_print_dropped']}")
    print(f"Dropped (unrepaired): {stats['unrepaired_dropped']}")
    print(f"Dropped (repair not possible): {stats['repair_not_possible_dropped']}")
    
    total_kept = stats['no_leak_kept'] + stats['uncertain_kept'] + stats['repaired_kept']
    total_dropped = stats['leak_print_dropped'] + stats['unrepaired_dropped'] + stats['repair_not_possible_dropped']
    print(f"\nTotal kept: {total_kept}")
    print(f"Total dropped: {total_dropped}")
    
    return stats


# Apply final decisions
final_stats = apply_final_decisions(candidates)


FINAL DECISION SUMMARY
Kept (no_leak): 10287
Kept (uncertain, flagged): 128
Kept (repaired): 0
Dropped (leak_print_or_comment): 581
Dropped (unrepaired): 3097
Dropped (repair not possible): 138

Total kept: 10415
Total dropped: 3816


In [ ]:
# ============================================================================
# SAVE OUTPUT (atomic final write)
# ============================================================================

def save_output(records: List[Dict[str, Any]], output_path: Path) -> None:
    """
    Save all records to output JSONL file atomically.
    Same number of records as input - modified in-place.
    """
    output_path.parent.mkdir(parents=True, exist_ok=True)
    tmp = output_path.with_suffix('.tmp')
    with open(tmp, 'w', encoding='utf-8') as f:
        for record in records:
            f.write(json.dumps(record, ensure_ascii=False) + "\n")
    tmp.rename(output_path)
    
    print(f"Saved {len(records)} records to {output_path}")


# Save output
save_output(all_records, OUTPUT_JSONL_PATH)

print(f"\nInput records: {len(all_records)}")
print(f"Output records: {len(all_records)} (same count - modified in-place)")

In [12]:
# ============================================================================
# SUMMARY STATISTICS
# ============================================================================

def compute_summary_stats(records: List[Dict[str, Any]], heuristic_stats: Dict, 
                          pass1_result: Dict, pass2_result: Dict,
                          validation_result: Dict, final_stats: Dict) -> None:
    """
    Compute and display comprehensive summary statistics.
    """
    print("\n" + "="*70)
    print("LEAK DETECTION PIPELINE - FINAL SUMMARY")
    print("="*70)
    
    # Count tiers in final output
    tier_counts = Counter()
    for record in records:
        tier = record.get("tier") or record.get("audit", {}).get("tier", "unknown")
        tier_counts[tier] += 1
    
    # Count records with leak_audit
    audited_count = sum(1 for r in records if "leak_audit" in r)
    
    # Count by leak label
    label_counts = Counter()
    for record in records:
        leak_audit = record.get("leak_audit", {})
        if leak_audit:
            pass1 = leak_audit.get("pass1", {})
            label = pass1.get("label", "unknown")
            label_counts[label] += 1
    
    # Count risk flags
    flag_counts = Counter()
    for record in records:
        flags = record.get("risk_flags", [])
        for flag in flags:
            flag_counts[flag] += 1
    
    print("\n--- PROCESSING SUMMARY ---")
    print(f"Total input records: {len(records)}")
    print(f"Records processed (tier-filtered): {heuristic_stats['processed']}")
    print(f"Candidates detected (answer in code): {heuristic_stats['candidates']}")
    print(f"Auto-dropped (direct print/return): {heuristic_stats['auto_drop_direct_print']}")
    print(f"LLM-reviewed candidates: {len(candidates)}")
    
    print("\n--- HEURISTIC BUCKET BREAKDOWN ---")
    for bucket, count in heuristic_stats['buckets'].most_common():
        print(f"  {bucket}: {count}")
    
    print("\n--- PASS 1 LABEL BREAKDOWN ---")
    for label, count in pass1_result.get('labels', {}).items():
        print(f"  {label}: {count}")
    
    print("\n--- REPAIR SUMMARY ---")
    print(f"  Repair attempted: {pass2_result.get('repaired', 0) + pass2_result.get('not_possible', 0)}")
    print(f"  Repair successful: {pass2_result.get('repaired', 0)}")
    print(f"  Repair not possible: {pass2_result.get('not_possible', 0)}")
    print(f"  Validation passed: {validation_result.get('validation_passed', 0)}")
    print(f"  Validation failed: {validation_result.get('validation_failed', 0)}")
    
    print("\n--- FINAL TIER DISTRIBUTION ---")
    for tier, count in tier_counts.most_common():
        pct = count / len(records) * 100
        print(f"  {tier}: {count} ({pct:.1f}%)")
    
    print("\n--- RISK FLAGS ADDED ---")
    leak_flags = [f for f in flag_counts if "leak" in f.lower()]
    for flag in leak_flags:
        print(f"  {flag}: {flag_counts[flag]}")
    
    print("\n--- FINAL DECISION BREAKDOWN ---")
    print(f"  Kept (no leak): {final_stats.get('no_leak_kept', 0)}")
    print(f"  Kept (uncertain): {final_stats.get('uncertain_kept', 0)}")
    print(f"  Kept (repaired): {final_stats.get('repaired_kept', 0)}")
    print(f"  Dropped (leak_print): {final_stats.get('leak_print_dropped', 0)}")
    print(f"  Dropped (unrepaired): {final_stats.get('unrepaired_dropped', 0)}")
    print(f"  Dropped (not possible): {final_stats.get('repair_not_possible_dropped', 0)}")
    print(f"  Dropped (auto, direct print): {heuristic_stats['auto_drop_direct_print']}")
    
    total_drops = (heuristic_stats['auto_drop_direct_print'] + 
                   final_stats.get('leak_print_dropped', 0) +
                   final_stats.get('unrepaired_dropped', 0) +
                   final_stats.get('repair_not_possible_dropped', 0))
    
    print(f"\n--- TOTAL IMPACT ---")
    print(f"  Total records dropped due to leaks: {total_drops}")
    print(f"  Total records repaired: {validation_result.get('validation_passed', 0)}")
    print(f"  Total records flagged (uncertain): {final_stats.get('uncertain_kept', 0)}")


# Compute summary
compute_summary_stats(all_records, heuristic_stats, pass1_result, pass2_result, 
                      validation_result, final_stats)


LEAK DETECTION PIPELINE - FINAL SUMMARY

--- PROCESSING SUMMARY ---
Total input records: 68191
Records processed (tier-filtered): 54389
Candidates detected (answer in code): 16753
Auto-dropped (direct print/return): 2522
LLM-reviewed candidates: 14231

--- HEURISTIC BUCKET BREAKDOWN ---
  comment_mentions: 6903
  other: 5236
  direct_print_or_return: 2522
  assert_equals: 2092

--- PASS 1 LABEL BREAKDOWN ---
  no_leak: 10287
  leak_assert_or_hardcode: 3107
  leak_print_or_comment: 581
  leak_answer_seeded: 128
  uncertain: 128

--- REPAIR SUMMARY ---
  Repair attempted: 3235
  Repair successful: 3097
  Repair not possible: 138
  Validation passed: 0
  Validation failed: 3097

--- FINAL TIER DISTRIBUTION ---
  core: 46613 (68.4%)
  drop: 20140 (29.5%)
  extended: 1438 (2.1%)

--- RISK FLAGS ADDED ---
  answer_leak_repair_failed: 3097
  answer_leak_direct_print: 2522
  answer_leak_print_or_comment: 581
  answer_leak_unrepaired: 138
  answer_leak_uncertain: 128

--- FINAL DECISION BREAKD

In [13]:
# ============================================================================
# SAMPLE INSPECTION - VIEW EXAMPLES OF EACH CATEGORY
# ============================================================================

def show_sample(records: List[Dict[str, Any]], filter_fn, title: str, n: int = 2) -> None:
    """Show sample records matching a filter."""
    matching = [r for r in records if filter_fn(r)]
    print(f"\n{'='*70}")
    print(f"{title} ({len(matching)} total, showing {min(n, len(matching))})")
    print("="*70)
    
    for i, record in enumerate(matching[:n]):
        print(f"\n--- Example {i+1} ---")
        print(f"ID: {record.get('id', 'N/A')}")
        print(f"Dataset: {record.get('dataset', 'N/A')}")
        print(f"Tier: {record.get('tier', record.get('audit', {}).get('tier', 'N/A'))}")
        
        leak_audit = record.get("leak_audit", {})
        if leak_audit:
            print(f"Heuristic bucket: {leak_audit.get('heuristic_bucket', 'N/A')}")
            pass1 = leak_audit.get("pass1", {})
            print(f"Label: {pass1.get('label', 'N/A')}")
            print(f"Confidence: {pass1.get('confidence', 'N/A')}")
            print(f"Reasons: {pass1.get('reasons', [])}")
            
            pass2 = leak_audit.get("pass2_repair", {})
            if pass2:
                print(f"Repair status: {pass2.get('status', 'N/A')}")
            
            validation = leak_audit.get("validation", {})
            if validation:
                print(f"Validation: {validation.get('passed', 'N/A')} - {validation.get('reason', '')}")
        
        flags = record.get("risk_flags", [])
        if flags:
            print(f"Risk flags: {flags}")
        
        # Show problem snippet
        problem_text = record.get("problem", {}).get("text", "")[:200]
        print(f"\nProblem: {problem_text}...")
        
        # Show code snippet
        pass_at_k = record.get("outcome", {}).get("pass_at_k", 0)
        attempts = record.get("attempts", [])
        if pass_at_k > 0 and pass_at_k <= len(attempts):
            code = attempts[pass_at_k - 1].get("code", "")[:300]
            print(f"\nCode: {code}...")


# Show examples of each category
show_sample(all_records, 
            lambda r: r.get("leak_audit", {}).get("pass1", {}).get("label") == "no_leak",
            "NO LEAK (kept)")

show_sample(all_records,
            lambda r: r.get("leak_audit", {}).get("pass1", {}).get("label") == "leak_print_or_comment",
            "LEAK: PRINT OR COMMENT (dropped)")

show_sample(all_records,
            lambda r: (r.get("leak_audit", {}).get("validation", {}).get("passed") == True),
            "REPAIRED SUCCESSFULLY (kept)")

show_sample(all_records,
            lambda r: "answer_leak_uncertain" in r.get("risk_flags", []),
            "UNCERTAIN (kept with flag)")


NO LEAK (kept) (10287 total, showing 2)

--- Example 1 ---
ID: numina1.5_40_20260203_041
Dataset: numina1.5:aops
Tier: core
Heuristic bucket: other
Label: no_leak
Confidence: 0.95
Reasons: ['The code derives the answer through computation rather than hardcoding or printing the expected value directly.', 'The expected answer does not appear as a literal constant in the code or comments.']

Problem: Suppose that $P(x)$ is a monic quadratic polynomial satisfying $aP(a) = 20P(20) = 22P(22)$ for some integer $a\neq 20, 22$. Find the minimum possible positive value of $P(0)$.

[i]Proposed by Andrew W...

Code: # Goal: Determine the minimal positive value of P(0) for a monic quadratic P(x)
# satisfying aP(a)=20P(20)=22P(22) with integer a≠20,22.

# Plan: Express P(x)=x^2+dx+e, derive d,e in terms of a from the symmetric
# condition using polynomial equality, enumerate admissible a, and choose
# minimal pos...

--- Example 2 ---
ID: numina1.5_17_20260203_018
Dataset: numina1.5:aops
Tier: core